MLNLP Project: Part II: Viterbi for Name Classification
--------------------------------------------

# Authors
    - Josue Happe
        - josue.happe.etu@univ-lille.fr
    - Selim Lakhdar
        - selim.lakhdar.etu@univ-lille.fr    

# Libraries

In [145]:
import numpy as np
import re
import pandas as pd
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Data

In [146]:
data = pd.read_csv("Ultimate_Gender_List.csv")
data.head()

,name,gender,score,Right_Name,Right_Gender,Frequency,Include?,probability
0,1,F,NaN,NaN,NaN,38.0,No,NaN
1,1,M,NaN,NaN,NaN,56.0,Yes,NaN
2,A,M,1.0,NaN,NaN,NaN,NaN,NaN
3,A'isha,F,0.0,NaN,NaN,NaN,NaN,NaN
4,A'ishah,F,0.0,NaN,NaN,NaN,NaN,NaN


In [147]:
# remove first 3 and last one
data = data[3:-1]

# On récupère les noms et les genres
X = np.array(data["name"])
y = np.array(data["gender"])

# On créé le vocabulaire
voc = []
for k in X:
    for c in k:
        if c not in voc:
            voc.append(c)

In [148]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

# Tagging (BIOES)

In [149]:
# On fait la liste des caractères et de leurs tags associés, on sépare chaque nom avec un espace
tags = []
liste_tags = ['O']

for i in range(len(X_train)):
    genre = y_train[i]
    
    # Soit on a un nom a une seule lettre (on tague S-genre)
    if (len(X_train[i])==1):
        if (X_train[i][0]!=' '):
            tags.append((X_train[i][0],'S-'+genre))
            t = 'S-'+genre
            if t not in liste_tags:
                liste_tags.append(t)
        else:
            tags.append((X_train[i][0],'O'))
    elif (len(X_train[i])>1):
        # On tag le premier cractère
        prec = ' '
        current = X_train[i][0]
        suivant = X_train[i][1]
        if (current==' '):
            tags.append((current,'O'))
        else:
            tags.append((current,'B-'+genre))
            t = 'B-'+genre
            if t not in liste_tags:
                liste_tags.append(t)
        for k in range(1,len(X_train[i])-1):
            prec = X_train[i][k-1]
            current = X_train[i][k]
            suivant = X_train[i][k+1]
            if (current==' '):
                tags.append((current,'O'))
            else:
                tags.append((current,'I-'+genre))
                t = 'I-'+genre
                if t not in liste_tags:
                    liste_tags.append(t)
        if (suivant==' '):
            tags.append((suivant,'O'))
        else:
            tags.append((suivant,'E-'+genre))
            t = 'E-'+genre
            if t not in liste_tags:
                liste_tags.append(t)
    tags.append((' ','O'))
    
print("len(tags):", len(tags), tags[:10])
print("len(liste_tags):", len(liste_tags), liste_tags)

len(tags): 695117 [('W', 'B-F'), ('a', 'I-F'), ('u', 'I-F'), ('n', 'I-F'), ('i', 'I-F'), ('t', 'I-F'), ('a', 'E-F'), (' ', 'O'), ('A', 'B-M'), ('a', 'I-M')]
len(liste_tags): 12 ['O', 'B-F', 'I-F', 'E-F', 'B-M', 'I-M', 'E-M', 'B-Unisex', 'I-Unisex', 'E-Unisex', 'S-F', 'S-M']


# Viterbi

In [150]:
# On prépare les matrices A et B pour définir notre HMM

# La matrice A définit les probabilité de transition d'un tag à un autre
A = np.zeros((len(liste_tags),len(liste_tags))).astype(float)

# La matrice B définit les probabilités d'observation'
B = np.zeros((len(liste_tags),len(voc))).astype(float)

for i in range(len(tags)-1):
    suivant = tags[i+1]
    current = tags[i]
    index_voc = voc.index(current[0])
    index_crt_tag = liste_tags.index(current[1])
    index_next_tag = liste_tags.index(suivant[1])
    B[index_crt_tag][index_voc] +=1
    A[index_crt_tag][index_next_tag] +=1
    
B[liste_tags.index(tags[-1][1])][voc.index(tags[-1][0])]

for i in range(len(A)):
    A[i] = A[i]/np.sum(A[i])
    B[i] = B[i]/np.sum(B[i])

In [151]:
# Algorithme de Viterbi
def viterbi(o,A,B,Pi,voc):
    T = len(o)
    N = len(A)
    Vit = np.zeros((N,T))
    backpointer = np.zeros((N,T)).astype(int)
    for s in range(N):
        Vit[s][0] = Pi[s]*B[s][voc.index(o[0])]
        backpointer[s][0]=0
    for t in range(1,T):
        for s in range(N):
            Vit[s][t] = max(Vit[:,t-1]*A[:,s]*B[s][voc.index(o[t])])
            backpointer[s][t] = np.argmax(Vit[:,t-1]*A[:,s]*B[s][voc.index(o[t])])
    bestpathprob = max(Vit[:,T-1])
    bestpathpointer = np.argmax(Vit[:,T-1])
    bestpath = []
    for i in range(T-1,-1,-1):
        bestpath.append(liste_tags[bestpathpointer])
        bestpathpointer = backpointer[bestpathpointer,i]
    return list(reversed(bestpath)), bestpathprob

In [152]:
# Une fonction donnant le genre dominant d'après une liste de tags
def gender_from_tags(tags):
    genders = {}
    for t in tags:
        if t == 'O':
            genders['O'] = genders.get('O',0)+1
        else:
            g = t.split('-')[1]
            genders[g] = genders.get(g,0)+1
    return max(genders, key=genders.get)

# Test

In [153]:
viterbi("James",A,B,np.ones(len(A))/len(A),voc)

(['B-M', 'I-M', 'I-M', 'I-M', 'I-M'], 6.948294893422862e-08)

In [154]:
gender_from_tags(viterbi("James",A,B,np.ones(len(A))/len(A),voc)[0])

'M'

# Pred

In [155]:
y_pred = []
n = len(X_test)

for i in range(n):
    if i % 5000 == 0:
        print(i, "/", n)
    
    y_pred.append(gender_from_tags(viterbi(X_test[i],A,B,np.ones(len(A))/len(A),voc)[0]))

0 / 30389
5000 / 30389
10000 / 30389
15000 / 30389
20000 / 30389
25000 / 30389
30000 / 30389


In [156]:
print(classification_report(y_test,y_pred, labels=['F', 'M', 'Unisex']))

              precision    recall  f1-score   support

           F       0.72      0.66      0.69     18344
           M       0.55      0.42      0.47     11783
      Unisex       0.02      0.36      0.04       262

   micro avg       0.56      0.56      0.56     30389
   macro avg       0.43      0.48      0.40     30389
weighted avg       0.65      0.56      0.60     30389



On remarque que le score obtenu pour le label Unisex est très faible, cela pourrait s'expliquer par le fait qu'il y ait très peu de noms epicènes dans nos données

In [157]:
np.unique(y_pred)

array(['F', 'M', 'O', 'Unisex'], dtype='<U6')

# Tagging (BIO)

In [158]:
# On fait la liste des caractères et de leurs tags associés, on sépare chaque nom avec un espace
tags = []
liste_tags = ['O']

for i in range(len(X_train)):
    genre = y_train[i]
    
    #On tag la première lettre d'un prénom avec B-genre, les suivantes avec I-genre
    nom = X_train[i]
    for k in range(len(nom)):
        if k==0:
            t = 'B-'+genre
        else:
            t = 'I-'+genre
        tags.append((nom[k],t))
        if t not in liste_tags:
            liste_tags.append(t)
    tags.append((' ','O'))
    
print("len(tags):", len(tags), tags[:10])
print("len(liste_tags):", len(liste_tags), liste_tags)

len(tags): 695117 [('W', 'B-F'), ('a', 'I-F'), ('u', 'I-F'), ('n', 'I-F'), ('i', 'I-F'), ('t', 'I-F'), ('a', 'I-F'), (' ', 'O'), ('A', 'B-M'), ('a', 'I-M')]
len(liste_tags): 7 ['O', 'B-F', 'I-F', 'B-M', 'I-M', 'B-Unisex', 'I-Unisex']


In [159]:
# On prépare les matrices A et B pour définir notre HMM

# La matrice A définit les probabilité de transition d'un tag à un autre
A = np.zeros((len(liste_tags),len(liste_tags))).astype(float)

# La matrice B définit les probabilités d'observation'
B = np.zeros((len(liste_tags),len(voc))).astype(float)

for i in range(len(tags)-1):
    suivant = tags[i+1]
    current = tags[i]
    index_voc = voc.index(current[0])
    index_crt_tag = liste_tags.index(current[1])
    index_next_tag = liste_tags.index(suivant[1])
    B[index_crt_tag][index_voc] +=1
    A[index_crt_tag][index_next_tag] +=1
    
B[liste_tags.index(tags[-1][1])][voc.index(tags[-1][0])]

for i in range(len(A)):
    A[i] = A[i]/np.sum(A[i])
    B[i] = B[i]/np.sum(B[i])

In [160]:
y_pred2 = []
n = len(X_test)

for i in range(n):
    if i % 5000 == 0:
        print(i, "/", n)
    
    y_pred2.append(gender_from_tags(viterbi(X_test[i],A,B,np.ones(len(A))/len(A),voc)[0]))

0 / 30389
5000 / 30389
10000 / 30389
15000 / 30389
20000 / 30389
25000 / 30389
30000 / 30389


In [161]:
print(classification_report(y_test,y_pred2, labels=['F', 'M', 'Unisex']))

              precision    recall  f1-score   support

           F       0.75      0.68      0.71     18344
           M       0.60      0.44      0.51     11783
      Unisex       0.02      0.44      0.04       262

   micro avg       0.58      0.58      0.58     30389
   macro avg       0.46      0.52      0.42     30389
weighted avg       0.69      0.58      0.63     30389



Les scores obtenus sont un peu meilleurs avec l'étiquettage BIO, testons avec l'étiquettage IO

# Tagging (IO)

In [162]:
# On fait la liste des caractères et de leurs tags associés, on sépare chaque nom avec un espace
tags = []
liste_tags = ['O']

for i in range(len(X_train)):
    genre = y_train[i]
    
    #On tag les lettres d'un prénom avec I-+genre
    nom = X_train[i]
    for k in range(len(nom)):
        t = 'I-'+genre
        tags.append((nom[k],t))
        if t not in liste_tags:
            liste_tags.append(t)
    tags.append((' ','O'))
    
print("len(tags):", len(tags), tags[:10])
print("len(liste_tags):", len(liste_tags), liste_tags)

len(tags): 695117 [('W', 'I-F'), ('a', 'I-F'), ('u', 'I-F'), ('n', 'I-F'), ('i', 'I-F'), ('t', 'I-F'), ('a', 'I-F'), (' ', 'O'), ('A', 'I-M'), ('a', 'I-M')]
len(liste_tags): 4 ['O', 'I-F', 'I-M', 'I-Unisex']


In [163]:
# On prépare les matrices A et B pour définir notre HMM

# La matrice A définit les probabilité de transition d'un tag à un autre
A = np.zeros((len(liste_tags),len(liste_tags))).astype(float)

# La matrice B définit les probabilités d'observation'
B = np.zeros((len(liste_tags),len(voc))).astype(float)

for i in range(len(tags)-1):
    suivant = tags[i+1]
    current = tags[i]
    index_voc = voc.index(current[0])
    index_crt_tag = liste_tags.index(current[1])
    index_next_tag = liste_tags.index(suivant[1])
    B[index_crt_tag][index_voc] +=1
    A[index_crt_tag][index_next_tag] +=1
    
B[liste_tags.index(tags[-1][1])][voc.index(tags[-1][0])]

for i in range(len(A)):
    A[i] = A[i]/np.sum(A[i])
    B[i] = B[i]/np.sum(B[i])

In [164]:
y_pred3 = []
n = len(X_test)

for i in range(n):
    if i % 5000 == 0:
        print(i, "/", n)
    
    y_pred3.append(gender_from_tags(viterbi(X_test[i],A,B,np.ones(len(A))/len(A),voc)[0]))

0 / 30389
5000 / 30389
10000 / 30389
15000 / 30389
20000 / 30389
25000 / 30389
30000 / 30389


In [165]:
print(classification_report(y_test,y_pred3, labels=['F', 'M', 'Unisex']))

              precision    recall  f1-score   support

           F       0.76      0.67      0.71     18344
           M       0.60      0.44      0.51     11783
      Unisex       0.02      0.44      0.04       262

    accuracy                           0.58     30389
   macro avg       0.46      0.52      0.42     30389
weighted avg       0.69      0.58      0.63     30389



On obtient quasiment les mêmes scores qu'avec l'étiquettage BIO

# En définissant un autre vocabulaire

Chaque lettre est représentée par 'consonne' 'voyelle' ou 'autre' dans le vocabulaire ("Robert" devient "cvcvcc")

In [166]:
#Une fonction qui convertit un nom en représentation "voyelle-consonne"
def vowel_consonant(nom):
    res = ""
    for k in nom:
        if re.search("[aeiou]",k):
            res+="v"
        elif re.search("[a-zA-Z]",k):
            res+="c"
        else:
            res+="a"
    return res

#On encode tous les noms de X_train et X_test selon cette représentation
X_train_vc = []
X_test_vc = []
for i in range(len(X_train)):
    X_train_vc.append(vowel_consonant(X_train[i]))
for j in range(len(X_test)):
    X_test_vc.append(vowel_consonant(X_test[j]))
X_train_vc = np.array(X_train_vc)
X_test_vc = np.array(X_test_vc)
print("X_train[:3]", X_train[:3])
print("X_train_vc[:3]", X_train_vc[:3])

X_train[:3] ['Waunita' 'Aarit' 'Jusin']
X_train_vc[:3] ['cvvcvcv' 'cvcvc' 'cvcvc']


In [167]:
# On fait la liste du type de caractère (dans voc2) et de leurs tags associés, on sépare chaque nom avec un espace
tags = []
liste_tags = ['O']
voc2 = ["v","c","a",' '] #v pour voyelle, c pour consonne, "a" pour autre, on garde l'espace pour séparer les prénoms

for i in range(len(X_train_vc)):
    genre = y_train[i]
    
    #On tag la première lettre d'un prénom avec B-genre, les suivantes avec I-genre
    nom = X_train_vc[i]
    for k in range(len(nom)):
        if k==0:
            t = 'B-'+genre
        else:
            t = 'I-'+genre
        tags.append((nom[k],t))
        if t not in liste_tags:
            liste_tags.append(t)
    tags.append((' ','O'))
    
print("len(tags):", len(tags), tags[:10])
print("len(liste_tags):", len(liste_tags), liste_tags)

len(tags): 695117 [('c', 'B-F'), ('v', 'I-F'), ('v', 'I-F'), ('c', 'I-F'), ('v', 'I-F'), ('c', 'I-F'), ('v', 'I-F'), (' ', 'O'), ('c', 'B-M'), ('v', 'I-M')]
len(liste_tags): 7 ['O', 'B-F', 'I-F', 'B-M', 'I-M', 'B-Unisex', 'I-Unisex']


In [168]:
# On prépare les matrices A et B pour définir notre HMM

# La matrice A définit les probabilité de transition d'un tag à un autre
A = np.zeros((len(liste_tags),len(liste_tags))).astype(float)

# La matrice B définit les probabilités d'observation'
B = np.zeros((len(liste_tags),len(voc2))).astype(float)

for i in range(len(tags)-1):
    suivant = tags[i+1]
    current = tags[i]
    index_voc = voc2.index(current[0])
    index_crt_tag = liste_tags.index(current[1])
    index_next_tag = liste_tags.index(suivant[1])
    B[index_crt_tag][index_voc] +=1
    A[index_crt_tag][index_next_tag] +=1
    
B[liste_tags.index(tags[-1][1])][voc2.index(tags[-1][0])]

for i in range(len(A)):
    A[i] = A[i]/np.sum(A[i])
    B[i] = B[i]/np.sum(B[i])

In [169]:
y_pred4 = []
n = len(X_test_vc)

for i in range(n):
    if i % 5000 == 0:
        print(i, "/", n)
    
    y_pred4.append(gender_from_tags(viterbi(X_test_vc[i],A,B,np.ones(len(A))/len(A),voc2)[0]))

0 / 30389
5000 / 30389
10000 / 30389
15000 / 30389
20000 / 30389
25000 / 30389
30000 / 30389


In [170]:
print(classification_report(y_test,y_pred4, labels=['F', 'M', 'Unisex'], zero_division=0))

              precision    recall  f1-score   support

           F       0.66      0.73      0.70     18344
           M       0.51      0.44      0.47     11783
      Unisex       0.00      0.00      0.00       262

    accuracy                           0.61     30389
   macro avg       0.39      0.39      0.39     30389
weighted avg       0.60      0.61      0.60     30389



Le score obtenu est un pe plus inférieur aux scores obtenus précédemment, cela peux s'expliquer par le vocabulaire trop peu varié

# Avec un autre tagging

In [172]:
#On va taguer les lettres avec C-, V-en fonction de si ce sont des voyelles ou nom, 
# On fait la liste des caractères et de leurs tags associés, on sépare chaque nom avec un espace
tags = []
liste_tags = ['O']

for i in range(len(X_train)):
    genre = y_train[i]
    nom = X_train[i]
    for k in range(len(nom)):
        t = vowel_consonant(nom[k])+'-'+genre
        tags.append((nom[k],t))
        if t not in liste_tags:
            liste_tags.append(t)
    tags.append((' ','O'))
    
print("len(tags):", len(tags), tags[:10])
print("len(liste_tags):", len(liste_tags), liste_tags)

len(tags): 695117 [('W', 'c-F'), ('a', 'v-F'), ('u', 'v-F'), ('n', 'c-F'), ('i', 'v-F'), ('t', 'c-F'), ('a', 'v-F'), (' ', 'O'), ('A', 'c-M'), ('a', 'v-M')]
len(liste_tags): 10 ['O', 'c-F', 'v-F', 'c-M', 'v-M', 'a-F', 'a-M', 'c-Unisex', 'v-Unisex', 'a-Unisex']


In [174]:
# On prépare les matrices A et B pour définir notre HMM

# La matrice A définit les probabilité de transition d'un tag à un autre
A = np.zeros((len(liste_tags),len(liste_tags))).astype(float)

# La matrice B définit les probabilités d'observation'
B = np.zeros((len(liste_tags),len(voc))).astype(float)

for i in range(len(tags)-1):
    suivant = tags[i+1]
    current = tags[i]
    index_voc = voc.index(current[0])
    index_crt_tag = liste_tags.index(current[1])
    index_next_tag = liste_tags.index(suivant[1])
    B[index_crt_tag][index_voc] +=1
    A[index_crt_tag][index_next_tag] +=1
    
B[liste_tags.index(tags[-1][1])][voc.index(tags[-1][0])]

for i in range(len(A)):
    A[i] = A[i]/np.sum(A[i])
    B[i] = B[i]/np.sum(B[i])

In [175]:
y_pred5 = []
n = len(X_test)

for i in range(n):
    if i % 5000 == 0:
        print(i, "/", n)
    
    y_pred5.append(gender_from_tags(viterbi(X_test[i],A,B,np.ones(len(A))/len(A),voc)[0]))

0 / 30389
5000 / 30389
10000 / 30389
15000 / 30389
20000 / 30389
25000 / 30389
30000 / 30389


In [176]:
print(classification_report(y_test,y_pred5, labels=['F', 'M', 'Unisex'], zero_division=0))

              precision    recall  f1-score   support

           F       0.76      0.67      0.71     18344
           M       0.61      0.45      0.52     11783
      Unisex       0.02      0.42      0.04       262

    accuracy                           0.59     30389
   macro avg       0.46      0.51      0.42     30389
weighted avg       0.70      0.59      0.63     30389

